In [32]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [20]:
ushmm_url = "https://collections.ushmm.org"

In [60]:
page_data = []
for page_num in range(1, 30):
    print(page_num)
    base_url = f"https://collections.ushmm.org/search/?f%5Bavailability%5D%5B%5D=transcript&f%5Bavailability%5D%5B%5D=english&f%5Bbrowse%5D%5B%5D=testimonies&page={page_num}&per_page=100"
    s = requests.get(base_url)
    soup = BeautifulSoup(s.content)
    documents = soup.find_all("div", {"class": "document"})
    for document in documents:
        subpage = ushmm_url+document.find("a")["href"]
        try:
            identifiers = document.find("div", {"class": "record-identifiers"}).text.strip().replace("|", ",")
        except:
            ValueError
            identifiers = "unknown"
        date = "Unknown"
        sc = "Unknown"
        try:
            date = document.find("dd", {"class": "blacklight-display_date"}).text.strip()
        except:
            AttributeError
        try:
            sc = document.find("dt", {"class": "blacklight-special_collection"}).text.strip()
        except:
            AttributeError
        page_data.append((identifiers, date, subpage))

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29


In [62]:
df = pd.DataFrame(page_data, columns=["identifiers", "date", "page"])
df

,identifiers,date,page
0,"Oral History , Accession Number: 2001.28 , RG ...",interview: 2001 February 20,https://collections.ushmm.org/search/catalog/i...
1,"Oral History , Accession Number: 2015.382.1 , ...",interview: 2014 November 30,https://collections.ushmm.org/search/catalog/i...
2,"Oral History , Accession Number: 2001.213.1 , ...",Unknown,https://collections.ushmm.org/search/catalog/i...
3,"Oral History , RG Number: RG-50.999.0419",interview: 2013 April 25,https://collections.ushmm.org/search/catalog/i...
4,"Oral History , RG Number: RG-50.030.0274",interview: 1995 August 18,https://collections.ushmm.org/search/catalog/i...
...,...,...,...
2891,"Oral History , Accession Number: 1993.A.0097.1...",interview: 1982 June,https://collections.ushmm.org/search/catalog/i...
2892,"Oral History , Accession Number: 2011.93 , RG ...",interview: 2011 April 21,https://collections.ushmm.org/search/catalog/i...
2893,"Oral History , RG Number: RG-50.999.0560",interview: 2016 May 25,https://collections.ushmm.org/search/catalog/i...
2894,"Oral History , RG Number: RG-50.999.0663",interview: 2018 July 11,https://collections.ushmm.org/search/catalog/i...


In [63]:
base_url

'https://collections.ushmm.org/search/?f%5Bavailability%5D%5B%5D=transcript&f%5Bavailability%5D%5B%5D=english&f%5Bbrowse%5D%5B%5D=testimonies&page=29&per_page=100'

In [65]:
df.to_csv("data/oral_testimony_urls.csv", index=False)

In [66]:
print(df.head(5).to_markdown())

|    | identifiers                                                             | date                        | page                                                   |
|---:|:------------------------------------------------------------------------|:----------------------------|:-------------------------------------------------------|
|  0 | Oral History , Accession Number: 2001.28 , RG Number: RG-50.030.0410    | interview: 2001 February 20 | https://collections.ushmm.org/search/catalog/irn512935 |
|  1 | Oral History , Accession Number: 2015.382.1 , RG Number: RG-50.106.0245 | interview: 2014 November 30 | https://collections.ushmm.org/search/catalog/irn185375 |
|  2 | Oral History , Accession Number: 2001.213.1 , RG Number: RG-50.969.0001 | Unknown                     | https://collections.ushmm.org/search/catalog/irn558424 |
|  3 | Oral History , RG Number: RG-50.999.0419                                | interview: 2013 April 25    | https://collections.ushmm.org/search/catalog/irn5

In [58]:
df.head(5)

,identifiers,date,special_collection,page
0,Oral History | Accession Number: 2001.28 | RG ...,interview: 2001 February 20,Special Collection:,https://collections.ushmm.org/search/catalog/i...
1,Oral History | Accession Number: 2015.382.1 | ...,interview: 2014 November 30,Special Collection:,https://collections.ushmm.org/search/catalog/i...
2,Oral History | Accession Number: 2001.213.1 | ...,Unknown,Special Collection:,https://collections.ushmm.org/search/catalog/i...
3,Oral History | RG Number: RG-50.999.0419,interview: 2013 April 25,Special Collection:,https://collections.ushmm.org/search/catalog/i...
4,Oral History | RG Number: RG-50.030.0274,interview: 1995 August 18,Special Collection:,https://collections.ushmm.org/search/catalog/i...


In [68]:
test = df.page.tolist()[0]
test

'https://collections.ushmm.org/search/catalog/irn512935'

In [76]:
def download_pdf(url, DIR):
    s =  requests.get(url)
    soup = BeautifulSoup(s.content)
    pdf = ""
    for a in soup.find_all("a"):
        if "pdf" in a["href"]:
            pdf = a["href"]
    if pdf != "":
        s = requests.get(pdf)
    filename = pdf.split("/")[-1]
    with open(f"{DIR}/{filename}", "wb") as f:
        f.write(s.content)
download_pdf(test, "pdfs")

https://collections.ushmm.org/oh_findingaids/RG-50.030.0410_trs_en.pdf
RG-50.030.0410_trs_en.pdf
